# Visualize and Analyze Data

This notebook will explore the data we downloaded in the `Customize and Access Data` notebook by performing the following steps:

1. Read in ATL07 data and plot against MODIS Corrected Reflectance (true color) imagery
2. Overlay ATL07 data over MODIS IST
3. Extract MODIS IST values along ICESat-2 heights
4. Extract ICESat-2 heights along MODIS IST values.
5. Plot MODIS IST and ICESat-2 heights
   

### Import packages

We'll only have to do these manual installs for the dry-run. Luis will get these installed prior to AGU!

In [ ]:
# Need to install `plotly` on top of Pangeo
!conda install -c plotly plotly=4.3.0 -y

# Need to instal `awscli` on top of Pangeo 
!conda install awscli -y
!conda list | grep pyproj

# Need to upgrade `pyproj` on this Pangeo environment. Had some issues getting this to install in our environment but this worked:
import sys
!{sys.executable} -m pip install pyproj==2.4.1

In [ ]:
#%run functions.py

In [ ]:
from collections import defaultdict

from osgeo import gdal, osr
import math
import h5py
import xarray as xr
import pyproj
import pandas as pd
from shapely.geometry import Point,Polygon

import pyresample as prs
import cartopy.crs as ccrs
from matplotlib import pyplot as plt
import hvplot.pandas  # Adds the 'hvplot' method to pandas objects
import hvplot.xarray  # Adds the 'hvplot' method to xarray objects
#import plotly.graph_objects as go
#from plotly.offline import iplot
import rasterio
import functions

# from pathlib import Path
import numpy as np
#import geopandas as gpd
from datetime import datetime, timedelta
from holoviews.operation.datashader import datashade
import datashader.geo

%matplotlib inline

### Load data from Amazon s3 bucket

We'll only have to do this manual s3 loading for the dry-run. Luis will get the outputs copied to the environment prior to AGU!

In [ ]:
!aws --no-sign-request --region=us-west-2 s3 sync s3://nsidc-agu-tutorial-data/ Outputs

## Read in our data

We will be using a few different packages to bring our data in...

* [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/getting_started/overview.html)
    * Pandas is built on top of NumPy and provides easy to work with data structures. The pandas dataframe is a 2-D tabular data structure with labeled axes. 
* [GeoPandas GeoDataFrame](http://geopandas.org/reference/geopandas.GeoDataFrame.html) 
    * A GeoDataFrame object is a pandas.DataFrame that has a column with geospatial capabilities.
* [Gdal](https://pypi.org/project/GDAL/)
    * Tools for programming and manipulating the Geospatial Data Abstraction Library (GDAL).
* [Rasterio](https://rasterio.readthedocs.io/en/stable/index.html)
    * Rasterio reads and writes raster formats including GeoTIFF.
* [Xarray](http://xarray.pydata.org/en/stable/index.html)
    * Inspired by Pandas, xarray introduces labels in the form of dimensions, coordinates and attributes on top of raw NumPy-like arrays, which allows for a more intuitive, more concise, and less error-prone developer experience. [GOOD FOR MULTIDIMENSIONAL DATA]

### Load ATL07 granules into dataframes

We will start by defining variables to extract and combine as a single dataframe:

In [ ]:
VARIABLES = {
    'ATL07': [
        '/gt1l/sea_ice_segments/delta_time',
        '/gt1l/sea_ice_segments/latitude',
        '/gt1l/sea_ice_segments/longitude',
        '/gt1l/sea_ice_segments/heights/height_segment_confidence',
        '/gt1l/sea_ice_segments/heights/height_segment_height',
        '/gt1l/sea_ice_segments/heights/height_segment_quality',
        '/gt1l/sea_ice_segments/heights/height_segment_surface_error_est',
        '/gt1l/sea_ice_segments/heights/height_segment_length_seg',
        '/gt2l/sea_ice_segments/delta_time',
        '/gt2l/sea_ice_segments/latitude',
        '/gt2l/sea_ice_segments/longitude',
        '/gt2l/sea_ice_segments/heights/height_segment_confidence',
        '/gt2l/sea_ice_segments/heights/height_segment_height',
        '/gt2l/sea_ice_segments/heights/height_segment_quality',
        '/gt2l/sea_ice_segments/heights/height_segment_surface_error_est',
        '/gt2l/sea_ice_segments/heights/height_segment_length_seg',
        '/gt3l/sea_ice_segments/delta_time',
        '/gt3l/sea_ice_segments/latitude',
        '/gt3l/sea_ice_segments/longitude',
        '/gt3l/sea_ice_segments/heights/height_segment_confidence',
        '/gt3l/sea_ice_segments/heights/height_segment_height',
        '/gt3l/sea_ice_segments/heights/height_segment_quality',
        '/gt3l/sea_ice_segments/heights/height_segment_surface_error_est',
        '/gt3l/sea_ice_segments/heights/height_segment_length_seg'
    ]
}

#### We will use the `load_icesat2_as_dataframe` function from our functions module to load points from our ATL07 files: [EXPLAIN VALUES OF EACH VARIAIBLE - 150 PHOTONS PER SEGMENT]

In [ ]:
# Load ATL07 granules and combine into single ATL07 dataframe

df_07_101341 = functions.load_icesat2_as_dataframe('./Outputs/processed_ATL07-01_20190323101341_12980201_002_01.h5', VARIABLES)
df_07_211343 = functions.load_icesat2_as_dataframe('./Outputs/processed_ATL07-01_20190323211343_13050201_002_01.h5', VARIABLES)

atl07_frames = [df_07_101341, df_07_211343]
df_atl07 = pd.concat(atl07_frames,ignore_index=True)

df_atl07

#### We will use the `convert_to_gdf` function from our functions module to convert our DataFrame with 'longitude' and 'latitude' columns to a GeoDataFrame:

In [ ]:
gdf_07_101341 = functions.convert_to_gdf(df_07_101341)
gdf_07_211343 = functions.convert_to_gdf(df_07_211343)
gdf_atl07 = functions.convert_to_gdf(df_atl07)

gdf_atl07

### Load MODIS Ice Surface Temperature file

There were 13 granules returned over our time and area of interest. We can use Earthdata Search to easily visualize the coverage of each granule:
[MODIS Earthdata Search link](https://search.earthdata.nasa.gov/search/granules?p=C1000001160-NSIDC_ECS&q=mod29&sb=140%2C72%2C153%2C80&m=80.14349326047419!-42.27686414753069!0!0!0!0%2C2&qt=2019-03-23T00%3A00%3A00.000Z%2C2019-03-23T23%3A59%3A59.999Z&tl=1557787959!4!)

Let's choose a single MODIS granule to be used in our analysis below. 

Out of the 13 returned, these all have full coverage over our study area:

* MOD29.A2019082.0810.006.2019082191746.hdf
* MOD29.A2019082.0630.006.2019082191633.hdf
* MOD29.A2019082.0455.006.2019082132032.hdf
* MOD29.A2019082.0945.006.2019082191513.hdf

And after viewing the browse images, this looks to have the best data coverage:

* MOD29.A2019082.0140.006.2019082132036.hdf


#### Now let's load our MOD29 file into xarray:

In [ ]:
# Define local filepath and open as xarray

filepath = './Outputs/MOD29_A2019082_0140_006_2019082132036_HEGOUT.hdf'

mod29 = xr.open_dataset(filepath)
mod29

### Load MODIS Corrected Reflectance (true color) image and MODIS IST image

GeoTIFFs were created using the NASA Worldview snapshot feature: </n> 

MODIS Corrected Reflectance: https://go.nasa.gov/2OjfDSa </n>

MODIS IST: https://go.nasa.gov/34fBnp8

We'll read in the projection information below to help inform our plotting projection parameters in the next step:

In [ ]:
# Read geotiff images from gdal 

gdal.UseExceptions()

fname_cr = './Outputs/snapshot-2019-03-23T00_00_00Z_polar.tiff'
fname_ist = './Outputs/snapshot-2019-03-23T00_00_00Z-IST.tiff'

ds_cr = gdal.Open(fname_cr)
ds_ist = gdal.Open(fname_ist)
#data = ds.ReadAsArray()
#gt = ds.GetGeoTransform()
proj_cr = ds_cr.GetProjection()
proj_ist = ds_ist.GetProjection()

inproj = osr.SpatialReference()
inproj.ImportFromWkt(proj_cr)

#Print proj4 strings for each GeoTIFF
proj4_cr = inproj.ExportToProj4()
print('MODIS True Color proj4:', proj4_cr)
inproj.ImportFromWkt(proj_ist)
proj4_ist = inproj.ExportToProj4()
print('MODIS IST proj4:',proj4_cr)

## Visualize ICESat-2 height values 

We can visualize the ICESat-2 track locations and height values overlayed with MODIS Corrected Reflectance and MODIS Ice Surface Temperature. We should see higher ATL07 heights along fast ice, with lower heights over thin ice and then increasing height over drift ice.  [YOU WILL GET A MESSAGE IF YOU FIRST DOWNLOADED THE COASTLINE] [add ist values and crop image] [CHANGE WIDTH / HEIGHTS]

In [ ]:
# Read in data using rasterio

fname_cr = './Outputs/snapshot-2019-03-23T00_00_00Z_polar.tiff'
fname_ist = './Outputs/snapshot-2019-03-23T00_00_00Z-IST.tiff'
source_ist = rasterio.open(fname_ist, 'r')
source_cr = rasterio.open(fname_cr, 'r')
bounds_cr = (source_cr.bounds.left, source_cr.bounds.right, \
          source_cr.bounds.bottom, source_cr.bounds.top)
bounds_ist = (source_ist.bounds.left, source_ist.bounds.right, \
          source_ist.bounds.bottom, source_ist.bounds.top)

In [ ]:
# Plot ICESat-2 over MODIS Corrected Reflectance
fig = plt.figure(figsize=(8, 10))
img = plt.imread(fname_cr)
plot_proj = ccrs.Stereographic(central_longitude=-45, central_latitude=90, 
                               false_easting=0, false_northing=0,
                               true_scale_latitude=70)
ax = plt.axes(projection=plot_proj)
plt.title('ATL07 height values\n'
          '03/23/2019')

# set a margin around the data
ax.set_xmargin(0.05)
ax.set_ymargin(0.10)

# add the image. Because this image was a tiff, the "origin" of the image is in the upper left corner
ax.imshow(img, origin='upper', extent=bounds_cr, transform=plot_proj)
ax.coastlines(resolution='50m', color='black', linewidth=1)
              
heights = ax.scatter(gdf_atl07.longitude, gdf_atl07.latitude, transform=ccrs.Geodetic(),
          c=gdf_atl07.height_segment_height,
          cmap='Reds', alpha=0.5) 

cbar = fig.colorbar(heights, ax=ax)
cbar.set_label('Height (m)')


# Plot ICESat-2 over MODIS IST
fig = plt.figure(figsize=(8, 10))
img = plt.imread(fname_ist)
plot_proj = ccrs.Stereographic(central_longitude=-45, central_latitude=90, 
                               false_easting=0, false_northing=0,
                               true_scale_latitude=70)

ax = plt.axes(projection=plot_proj)
plt.title('ATL07 height values\n'
          '03/23/2019')

# set a margin around the data
ax.set_xmargin(0.05)
ax.set_ymargin(0.10)

# add the image. Because this image was a tiff, the "origin" of the image is in the upper left corner
ax.imshow(img, origin='upper', extent=bounds_ist, transform=plot_proj)
ax.coastlines(resolution='50m', color='black', linewidth=1)

heights = ax.scatter(gdf_atl07.longitude, gdf_atl07.latitude, transform=ccrs.Geodetic(),
          c=gdf_atl07.height_segment_height,
          cmap='Reds', alpha=0.5)              

cbar = fig.colorbar(heights, ax=ax)
cbar.set_label('Height (m)')
plt.show()


#### How does this look in 3d?

Coming soon...

... maybe not necessary...


In [ ]:
# From OpenAltimetry notebook:

# # Paste the OpenAltimetry selection parameters here
# OA_REFERENCE_URL = 'minx=72.0476&miny=36.6165&maxx=72.0505&maxy=36.6214&date=2019-05-02&trackId=516'
# # We populate a list with the photon data using the OpenAltimetry API, no HDF! 
# OA_URL = 'https://openaltimetry.org/data/icesat2/getPhotonData?client=jupyter&' + OA_REFERENCE_URL

# OA_PHOTONS = ['Noise', 'Low', 'Medium', 'High']
# # OA_PLOTTED_BEAMS = [1,2,3,4,5,6] you can select up to 6 beams for each ground track.
# # Some beams may not be usable due cloud covering or QC issues.
# OA_BEAMS = [3,4]

# df_beams = []
# oa_plots = []

# # This function will request the 6 tracks using OpenAltimetry's API
# def getPhotonData():
#     series = []
#     beam_index = 0;
#     for beam in OA_BEAMS:
#         beam_index += 1
#         b_url = OA_URL + '&beam=' + str(beam)
#         print(b_url)
#         print('Requesting Beam {}'.format(beam_index))
#         r = requests.get(b_url)
#         data = r.json()
#         series.append(data)
#     print ('Data requests completed you can now plot it on the next cell...')
#     return series

        
# photon_cloud = getPhotonData()

# track_index = 0
# for track in photon_cloud:
#     track_index +=1
#     for photons in track['series']:
#         if any(word in photons['name'] for word in OA_PHOTONS):
#             series = []
#             for p in photons['data']:
#                 series.append({
#                     'lat': p[0],
#                     'lon': p[2],
#                     'h': p[1]
#                 })
#             if (len(series) > 0):
#                 df = pd.DataFrame.from_dict(series)
#                 df.name = 'Beam ' + str(track_index) + ' ' + photons['name']
#                 df_beams.append(df) 

# for df in df_beams:
#     oa_plots.append(go.Scatter3d(name=df.name, x=df['lat'], y=df['lon'], z=df['h'], mode='markers',
#                                     marker=dict(
#                                         size=1,
#                                         color=df['h'],
#                                         colorscale='Viridis',   # choose a colorscale
#                                         opacity=0.8
#                                     )
#                                 )
#                    )
    
    
# layout = go.Layout(
#     width=900,
#     height=600,
#     scene = dict(aspectmode = 'manual', aspectratio =dict(x=1, y=1, z=0.5),
#                  xaxis=dict(title='Latitude'), yaxis=dict(title='Longitude'), zaxis=dict(title='Elevation (m)'))
# )

# fig = go.Figure(data=oa_plots, layout=layout)  

# iplot(fig)


# # Now we can get a histogram of the beam's elevation and their description
# for df in df_beams:
#     print(df.describe())
#     df.hist('h', bins=128)
    

### Overlay ATL07 values over MODIS Ice Surface Temperature



Before we plot the ATL07 heights over MODIS IST, we need to convert the Ice_Surface_Temperature values to kelvins. According to the [MOD29 User Guide](https://nsidc.org/data/mod29#title11) the Ice_Surface_Temperature (IST) values are stored as calibrated data. To convert to kelvins, use scale_factor = 0.01 and add_offset = 0.0 in the following equation:

IST = scale_factor × (calibrated data - add_offset)

The valid range for ISTs is 210 K to 313.20 K.

Let's apply this conversion to our xarray:

In [ ]:
# Apply calibrated data to kelvin conversion

mod29['Ice_Surface_Temperature'] = mod29['Ice_Surface_Temperature']*(0.01)

Plot MOD29 using Holoviews

(Provide quick summary of Holoviews)

In [ ]:
# title = 'MOD29 Ice Surface Temperature (K)'
rasterized_ist = mod29.Ice_Surface_Temperature.hvplot.image(rasterize=True, 
                                         cmap='viridis', 
                                         height=400, width=800).redim.range(Ice_Surface_Temperature=(240,270))
rasterized_ist.options(cmap='viridis')

# atl07_points = datashade(df_atl07.hvplot.points(x='longitude', y='latitude', c='mag', cmap=mag_cmap))
# ist_and_height = (rasterized_ist * atl07_points).relabel(title)
# ist_and_height

#Original plot from Bruce
# img = mod29.Ice_Surface_Temperature.hvplot.image().redim.range(Ice_Surface_Temperature=(240, 270))
# img.options(cmap='viridis')


#Incorporate ICESat-2 values
# datashade(df_atl07.hvplot.points(x='longitude', y='latitude'))
# x, y = datashader.geo.lnglat_to_meters(df_atl07.longitude, df_atl07.latitude)

## Extract MODIS IST values along the ICESat-2 tracks

Now that we've visually inspected height and IST values, let's do some analysis. We will now extract MODIS IST values at each ATL07 point.


### First we'll convert ATL07 'delta_time' to datetime values

In [ ]:
EPOCH = datetime(2018, 1, 1, 0, 0, 0)

def convert_delta_time(delta_time):
    '''
    Convert ICESat-2 'delta_time' parameter to UTC datetime
    '''
    utc_datetime = EPOCH + timedelta(seconds=delta_time)

    return utc_datetime

In [ ]:
# Add datetime values to each geopandas dataframe

gdf_07_101341['utc_datetime'] = gdf_07_101341['delta_time'].apply(convert_delta_time)
gdf_07_101341 = gdf_07_101341.drop(columns='delta_time')

gdf_07_211343['utc_datetime'] = gdf_07_211343['delta_time'].apply(convert_delta_time)
gdf_07_211343 = gdf_07_211343.drop(columns='delta_time')

gdf_atl07['utc_datetime'] = gdf_atl07['delta_time'].apply(convert_delta_time)
gdf_atl07 = gdf_atl07.drop(columns='delta_time')

gdf_atl07

### Compute along track distance

The `height_segment_length_seg` variable provides the length of each segment. We need these values to be additive so that we can plot against distance.  

In [ ]:
add_dist = [gdf_07_101341.height_segment_length_seg.values[0]]
for i in range(1, len(gdf_07_101341.index)):
    add_dist.append(add_dist[i-1]+gdf_07_101341.height_segment_length_seg.values[i])
gdf_07_101341['add_dist'] = add_dist

add_dist = [gdf_07_211343.height_segment_length_seg.values[0]]
for i in range(1, len(gdf_07_211343.index)):
    add_dist.append(add_dist[i-1]+gdf_07_211343.height_segment_length_seg.values[i])
gdf_07_211343['add_dist'] = add_dist
gdf_07_211343

### Convert back to pandas dataframe

Have both for good measure (different plotting api's mainly)

In [ ]:
df_07_101341 = pd.DataFrame(gdf_07_101341.drop(columns='geometry'))
df_07_211343 = pd.DataFrame(gdf_07_211343.drop(columns='geometry'))

### Interpolating MOD29 onto ICESat-2 points

To easily interpolate MOD29 to ICESat-2 points, we can use the `pyresample` library. The `radius_of_influence` parameter determines maximum radius to look for nearest neighbor interpolation.

In [ ]:
icesat2_geometry_101341 = prs.geometry.SwathDefinition(lons=df_07_101341['longitude'], lats=df_07_101341['latitude'])
icesat2_geometry_211343 = prs.geometry.SwathDefinition(lons=df_07_211343['longitude'], lats=df_07_211343['latitude'])

mod29_geometry = prs.geometry.SwathDefinition(lons=mod29['Longitude'], lats=mod29['Latitude'])
df_07_101341['mod29_ice_surface_temperature'] = prs.kd_tree.resample_nearest(mod29_geometry, mod29['Ice_Surface_Temperature'].values, icesat2_geometry_101341, radius_of_influence=1000, fill_value=np.nan)
df_07_211343['mod29_ice_surface_temperature'] = prs.kd_tree.resample_nearest(mod29_geometry, mod29['Ice_Surface_Temperature'].values, icesat2_geometry_211343, radius_of_influence=1000, fill_value=np.nan)
df_07_211343

# apply nan's to MOD29 land mask

# use .loc method

mask_101341 = df_07_101341['mod29_ice_surface_temperature'] == 50
df_07_101341['mod29_ice_surface_temperature'][mask_101341] = np.nan

mask_211343 = df_07_211343['mod29_ice_surface_temperature'] == 50
df_07_211343['mod29_ice_surface_temperature'][mask_211343] = np.nan

## Plot MODIS IST and ICESat-2 height along tracks

* This should show cold IST over the fast ice with fairly flat and higher ICESat-2 heights
* Warm IST will be seen in open water, thin ice just off the fast ice boundary
    * IST will decrease along transect
    * IS-2 should show low heights, but probably low quality (at least for ATL10, if any are available)
    
    
    
 [JUST PLOT ONE BEAM AT A TIME WITH DISTANCE]
 [ATL07 VS ATL10]
 [PLOT THOSE QUALITY VALUES - INTERACTIVE]
 [PLOT ATL10 QUALITY FLAGS LOWER ON ATL07]
 [TRY DIFFERENT BEAM]
 
 
 [dont need GEOPANDAS - JUST USE PANDAS]
 [SAME PLOT MAP TOOLS]
 1. SHOW MAP, 2. SHOW LAYERS, 3. XY PLOT, 

### First, we'll plot against time.

ATL07-01_20190323211343_13050201_002_01 is moving from low to high latitude: The start time begins at the fast ice, skips over an island, and moves towards warmer thin ice onto thicker drift ice.

ATL07-01_20190323101341_12980201_002_01 is moving from high to low latitude: The start time begins at the thicker drift ice, moving towards thinner ice and open water, ending at colder fast ice. 

In [ ]:
# Using pandas plotting (matplotlib backend)
fig, ax = plt.subplots(figsize=(15,5))
df_07_211343.plot(x='utc_datetime', y='height_segment_height', ax=ax)
df_07_211343.plot(x='utc_datetime', y='mod29_ice_surface_temperature', ax=ax, secondary_y=True)
plt.title('ATL07-01_20190323211343_13050201_002_01 unfiltered');

fig, ax = plt.subplots(figsize=(15,5))
df_07_101341.plot(x='utc_datetime', y='height_segment_height', ax=ax)
df_07_101341.plot(x='utc_datetime', y='mod29_ice_surface_temperature', ax=ax, secondary_y=True)
plt.title('ATL07-01_20190323101341_12980201_002_01 unfiltered');

### Plot MODIS IST vs. ICESat-2 as a scatterplot

* Over fast ice: IST should be close to constant and height will also be fairly constant
* Over drift ice, where it goes from thin ice toward thicker ice, we should see height increasing as IST decreases
* If we use IST = 271.2 K for open water and then find the IST at the first valid ATL10 value, we have the IST value at that freeboard. Then can use that relationship to get thin ice freeboard where there is ATL07 data, but not ATL10
* Comparing data with and without quality flag, the high temps are getting filtered out - more open water.

In [ ]:
datashade(df_07_101341.hvplot.points(x='mod29_ice_surface_temperature', y='height_segment_height', c='latitude', height=800, width=2000))


In [ ]:
datashade(df_07_211343.hvplot.points(x='mod29_ice_surface_temperature', y='height_segment_height', c='latitude', height=800, width=2000))

Add ATL10 to show coverage and freeboard estimate.

### Data filtering by quality flag

We'll compare data before and after applying the quality flag. For each beam group within each file, filter `/sea_ice_segments/heights/height_segment_height` based on `/sea_ice_segments/heights/height_segment_quality`. According to the [User Guide](https://nsidc.org/data/ATL07#title38):

height_segment_quality: a binary indicator (1 = good, 0 = bad) of segment quality

In [ ]:
qual_filter_101341 = df_07_101341['height_segment_quality']==1
qual_filter_211343 = df_07_211343['height_segment_quality']==1

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
df_07_211343[qual_filter_211343].plot(x='utc_datetime', y='height_segment_height', ax=ax)
df_07_211343[qual_filter_211343].plot(x='utc_datetime', y='mod29_ice_surface_temperature', ax=ax, secondary_y=True)
plt.title('ATL07-01_20190323211343_13050201_002_01 filtered');


# Using pandas plotting (matplotlib backend)
fig, ax = plt.subplots(figsize=(15, 5))
df_07_101341[qual_filter_101341].plot(x='utc_datetime', y='height_segment_height', ax=ax)
df_07_101341[qual_filter_101341].plot(x='utc_datetime', y='mod29_ice_surface_temperature', ax=ax, secondary_y=True)
plt.title('ATL07-01_20190323101341_12980201_002_01 filtered');

In [ ]:
datashade(df_07_101341[qual_filter_101341].hvplot.points(x='mod29_ice_surface_temperature', y='height_segment_height', c='latitude', height=800, width=2000))

In [ ]:
atl07_frames = [df_07_101341, df_07_211343]
df_atl07 = pd.concat(atl07_frames,ignore_index=True)
datashade(df_atl07.hvplot.points(x='mod29_ice_surface_temperature', y='height_segment_height', c='latitude', height=800, width=2000))

In [ ]:
datashade(df_07_211343[qual_filter_211343].hvplot.points(x='mod29_ice_surface_temperature', y='height_segment_height', c='latitude', size=400))

### Now plot against distance

This needs to be plotted by beam... 

## Extract ICESat-2 heights along MODIS IST values

* Use MODIS footprint as the baseline and match/bin IS-2 to each MODIS observation

In [ ]:
# # separate each beam

# gt1l = df_single['beam']=='gt1l'
# gt2l = df_single['beam']=='gt2l'
# gt3l = df_single['beam']=='gt3l'

In [ ]:
# fig, ax = plt.subplots(figsize=(15, 5))
# df_single[gt1l].plot(x='distance', y='height_segment_height', ax=ax)
# df_single[gt1l].plot(x='distance', y='mod29_ice_surface_temperature', ax=ax, secondary_y=True)

# fig, ax = plt.subplots(figsize=(15, 5))
# df_single[gt2l].plot(x='distance', y='height_segment_height', ax=ax)
# df_single[gt2l].plot(x='distance', y='mod29_ice_surface_temperature', ax=ax, secondary_y=True)

# fig, ax = plt.subplots(figsize=(15, 5))
# df_single[gt3l].plot(x='distance', y='height_segment_height', ax=ax)
# df_single[gt3l].plot(x='distance', y='mod29_ice_surface_temperature', ax=ax, secondary_y=True)

In [ ]:
df_single[qual_filter].plot.scatter(x='mod29_ice_surface_temperature', y='height_segment_height')



In [ ]:
## add ATL10 now? This could show coverage and show the freeboard estimate. 

### Add AMSR2 (AU_SI6) 89V-H difference to MODIS IST and ICESat-2 plots

* Difference for open water/thin ice large (~40-50 K), for thicker ice near zero (<10 K)
* Or calculate the polarization ratio (PR) = (89V – 89H)/(89V + 89H), which is commonly used in the PM sea ice community
* In theory can get concentration from PR (or 89 difference), but don’t really need to do that here.


### A couple other possibilities:
* Compare cross-over points between tracks 1298 and 1305. 
    * This is over drift ice, but might be able to correlate between them
    * Using all strong beams, there are 9 cross-over points, so there is 
* Compare 1298 repeat track from 12/22/2018 and 3/23/2019
    * For fast ice, should see an increase in height from Dec to Mar as snow depth and ice thickness increases
        * From a quick look at OA, I’m not really seeing this though
        * Not much from ATL10
        * Lots of data in ATL07, but don’t really see a connection – in fact Dec looks lower than Mar; this could be due to corrections/quality issues

### Extra Credit

#### Explore ATL03 (photon-level) heights

Load ATL03 Variables:

In [ ]:
VARIABLES = {
    'ATL03': [
        '/gt1l/heights/delta_time',
        '/gt1l/heights/h_ph',
        '/gt1l/heights/lat_ph',
        '/gt1l/heights/lon_ph',
        '/gt1l/heights/signal_conf_ph',
        '/gt2l/heights/delta_time',
        '/gt2l/heights/h_ph',
        '/gt2l/heights/lat_ph',
        '/gt2l/heights/lon_ph',
        '/gt2l/heights/signal_conf_ph',
        '/gt3l/heights/delta_time',
        '/gt3l/heights/h_ph',
        '/gt3l/heights/lat_ph',
        '/gt3l/heights/lon_ph',
        '/gt3l/heights/signal_conf_ph'
    ],

Use the load_icesat2_as_dataframe function from our functions module to load points from our ATL03 files:

In [ ]:
# Running out of memory on these:
# #df_03_103438 = functions.load_icesat2_as_dataframe('./Outputs/processed_ATL03_20190323103438_12980204_002_01.h5', VARIABLES)
# df_03_103947 = functions.load_icesat2_as_dataframe('./Outputs/processed_ATL03_20190323103947_12980205_002_01.h5', VARIABLES)
# df_03_212915 = functions.load_icesat2_as_dataframe('./Outputs/processed_ATL03_20190323212915_13050203_002_01.h5', VARIABLES)
# #df_03_213440 = functions.load_icesat2_as_dataframe('./Outputs/processed_ATL03_20190323213440_13050204_002_01.h5', VARIABLES)


#atl03_frames = [df_03_103438, df_03_103947, df_03_212915, df_03_213440]
#atl03_df = pd.concat(atl03_frames,ignore_index=True)


In [ ]:
# min_lon, max_lon = mod29.Longitude.min(), mod29.Longitude.max()
# min_lat, max_lat = mod29.Latitude.min(), mod29.Latitude.max()
# mod29_boundary = Polygon(
#     [
#         (min_lon, min_lat), 
#         (max_lon, min_lat), 
#         (max_lon, max_lat), 
#         (min_lon, max_lat), 
#         (min_lon, min_lat)
#     ]
# )

In [ ]:
# Warning: rather slow operation here, takes a few minutes for 200000 points
#distance = gdf_all.groupby(['filename', 'beam']).apply(functions.compute_along_track_distance)

# Warning: rather slow operation here, takes a few minutes for 200000 points
#distance_single = gdf_single.groupby(['filename','beam']).apply(functions.compute_along_track_distance)

# Convert pandas series to list
#distance_list = distance_single.tolist()

# Create new gdf with distance added
# gdf_dist = gdf_single.assign(distance = distance_list)